# core

> Config package for python

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export

import yaml
import os
import pathlib
import inspect
import traceback
import warnings


In [ ]:
#| exporti
def get_env(
        var:str, # The environment variable to get
        default:str = "default" # The default value to return if the environment variable is not set
    ):
    "Get a value from the environment variables."
    return os.environ.get(var, "default")


assert get_env('TEST') == 'default'
os.environ['TEST'] = 'test'
assert get_env('TEST') == 'test'


In [ ]:
#| exporti

def find_config_file(file, depth = 3):
    sources = {
        inspect.stack()[0][1],
        os.path.dirname(traceback.extract_stack()[-depth].filename),
        os.getcwd(),
        os.path.abspath(""),
    }

    file_exists = False
    for source in sources:
        filename = pathlib.Path(source, file)
        if os.path.exists(filename):
            file_exists = True
            # print(f"Found file {file} with source {source}")
            break

        else:
            # print(f"File {file} not found with source {source}")
            pass

    if not file_exists:
        source_locs = '\n - '.join(sources)
        raise FileNotFoundError(f"File {file} not found in any of these locations: \n - {source_locs}")
    
    return filename

# find_config_file("config.yaml")

Path('c:/Users/apdev/Documents/github/chronicle/py-config/nbs/config.yaml')

In [ ]:
#| exporti
def expr_constructor(loader, node):
    value = loader.construct_scalar(node)
    z = None
    try:
        z = eval(value)
    except:
        # import warnings
        def format_warning(message, category, filename, lineno, file=None, line=None):
            return ' %s:%s:\n %s:%s' % (filename, lineno, category.__name__, message)
        warnings.formatwarning = format_warning
        msg = f" Cannot evaluate expression in config.yaml: `{value}`"
        warnings.warn(msg, stacklevel=0)
    
    return z


def read_yaml(file):
    yaml.add_constructor('!expr', expr_constructor)
    return yaml.load(file, Loader=yaml.Loader)




In [ ]:
#| hide
x = read_yaml(
    """
    trials: 1
    expr: !expr os.getcwd()
    # expr2: !expr invalid()
    """
)
assert x['trials'] == 1
assert x['expr'] == os.getcwd()

In [ ]:
#| export

def config_get(
        value: str = None, # Name of value (None to read all values)
        py_config_active:str = None, # Name of configuration to read from. Defaults to the value of the `R_CONFIG_ACTIVE` environment variable ("default" if the variable does not exist).
        file:str = 'config.yaml', # Configuration file to read from.
        encoding:str = None
    ):
    "Get a value from the `config.yaml` file.Read from the currently active configuration, retrieving either a single named value or all values as a list."
    if py_config_active is None:
        py_config_active = get_env('R_CONFIG_ACTIVE', 'default')
    
    filename = find_config_file(file)

    dir_path = os.getcwd()
    file = pathlib.Path(dir_path, filename)

    with open(file, 'r', encoding = encoding) as stream:
        conf = read_yaml(stream)
    
    if value is None:
        return conf[py_config_active]
    else:
        return conf[py_config_active][value]


In [ ]:
assert config_get('trials') == 5
assert config_get('trials', 'production') == 30



In [ ]:
#| hide
test_fail(
    lambda: config_get('trials', file = "nofile.yaml"),
    contains = "File"
)


In [ ]:

config_get(py_config_active = "production")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()